# 8章 固有表現抽出

本章では、文章から人名や組織名といった固有名詞、日付などの時間表現、金額などの数値表現を抽出し、それぞれの固有表現のカテゴリーを判断する、固有表現抽出(Named Entity Recognition: NER)を扱う。<br>
一般的な固有表現のカテゴリーの定義としては、

- IREX(ニューヨーク大学): https://nlp.cs.nyu.edu/irex/index-j.html
- 拡張固有表現階層(理化学研究所): http://ene-project.info

などが挙げられる。よく用いられるIREX-NEの定義では、固有表現は以下の通りである。

- ORGANIZATION(組織名)
- 固有物名
- PERSON(人名)
- LOCATION(地名)
- DATE(日付表現)
- TIME(時間表現)
- MONEY(金額表現)
- PERCENT(割合表現)

固有表現抽出は、ユーザの投稿した文章から自動でタグ付けをする、個人情報保護のため文章中の人名にマスクをする、といったタスクに用いられる。

## 8.3 文字列の正規化

半角と全角といった実質的に同じ文字を統一する処理を「正規化」という。Pythonでは`unicodedata.normalize`関数が提供されている。本章では、これを用いて事前に正規化する。

In [2]:
# 8-3

import unicodedata

# 'NFKC'は正規化のモード
print(unicodedata.normalize('NFKC', "ＡＢＣ"))

ABC


## 8.5 IO法を用いたタグ付けと実装

## 8.5.1 IO法によるタグ付け

トークンへのタグ付けとして用いられるIO(Inside / Outside)法を実装する。IO法は、以下のようにタグ付けされる。

|トークン|タグ|
|-|:-:|
|A|I-(人名)|
|さん|O|
|は|O|
|BC|I-(組織名)|
|##D|I-(組織名)|
|株式会社|I-(組織名)|
|を|O|
|起業|O|
|し|O|
|た|O|
|。|O|

タグの`I-()`が連続している部分は、`##`を消した上で連結する。

一方、IO法は「日米」という単語をそれぞれの国でなく日米というひとつの固有表現として認識する。そのため、IO法を発展させた「BIO法」を後に実装する。

## 8.5.2 トークナイザ

固有表現を抽出する際には、文章と固有穂湯源からタグ列を作成したり、BERTが出力したタグ列から文章中の固有表現を得る関数を実装する必要がある。
本項では

- 文章と固有表現が与えられたときに符号化とタグ列の生成を行い、モデルに入力する関数(学習)
- 文章を符号化するとともに、各トークンの文章中のイチを特定する関数(推論)
- 文章とタグ列と各区トークンの文章中のイチが与えられたときに、文章中に含まれる固有表現に対応する文字列やイチを特定する関数(推論)

を実装する。

In [3]:
# 8-3

import itertools
import random
import json
import numpy as np
import unicodedata

import torch
from torch.utils.data import DataLoader
from transformers import BertJapaneseTokenizer, BertForTokenClassification

`transformers.BertJapaneseTokenizer`を拡張して、`NER_tokenizer`クラスを実装する。

In [4]:
class NER_tokenizer(BertJapaneseTokenizer):
    # 子クラスに def __init__()がない場合、
    # 子クラスの引数は親クラスのコンストラクタに引き継がれる
    def encode_plus_tagged(
        self,
        text: str,
        entities: list,
        max_length: int
    ) -> dict:
        """
        文章の符号化とラベル列の生成を行う
        トークン化で固有表現の間違った分割がされないように、
        固有表現で文章を分割->トークン化の順に処理するお
        """
        # text = '昨日のみらい事務所との打ち合わせは順調だった。'
        # entities = [{'name': 'みらい事務所', 'span': [3,9], 'type_id': 1}]
        # entities[0]['type_id'] = 1 は組織名を表す
        entities = sorted(entities, key=lambda x: x['span'][0])

        splitted_text_lst = []
        position = 0
        for entity in entities:
            start = entity['span'][0]
            end = entity['span'][1]
            label = entity['type_id']
            # position(初期値0)->entityのstartまでは固有表現でない文字列となる
            splitted_text_lst.append({"text": text[position: start], "label": 0})
            # start->endまでは固有表現なのでlabelをそのままlabelに当てはめる
            splitted_text_lst.append({"text": text[start: end], "label": label})

            position = end

        # entitiesの最後の要素より後はすべて固有表現でない文字列
        splitted_text_lst.append({"text": text[position:], "label": 0})
        # 長さ0の文字列(s['text'] is False)を除く
        splitted_text_lst = [s for s in splitted_text_lst if s['text']]

        tokens = [] # トークン化した文字列を追加
        labels = [] # splitted_text['label']を追加
        for t in splitted_text_lst:
            text = t['text']
            label = t['label']
            # tokenizer.tokenizeは符号化まではしない
            # splited_textの全要素をトークン化->後でまとめて符号化するため
            # とりあえずトークン化しておく
            tokens_tmp = self.tokenize(text)
            labels_tmp = [label] * len(tokens_tmp) # tokenの長さ分だけ延長する
            tokens.extend(tokens_tmp) # .extend()メソッドと同じ
            labels.extend(labels_tmp) # listを延長する

        # トークンのID化
        input_ids = self.convert_tokens_to_ids(tokens)
        # トークンIDを符号化する
        encoding = self.prepare_for_model(
            input_ids,
            max_length=max_length,
            padding='max_length',
            truncation=True,
        )
        # [CLS]と[SEP]トークンのラベルを0にする
        # entities引数には1文のみ入ってるため文中の[SEP]は無視できる？
        labels = [0] + labels[:max_length-2] + [0]
        # [PAD]トークンのラベルを0にする
        # text後部の穴埋め
        labels = labels + [0] * (max_length - len(labels))
        encoding["labels"] = labels
        return encoding

    def encode_plus_untagged(
        self,
        text: str,
        max_length: int = None,
        return_tensors: str = None,
    ) -> (dict, list):
        """
        文章をトークン化し、それぞれのトークンと文章中の文字列を対応付ける
        推論時にトークンごとのラベルを予測し、最終的に固有表現に変換する
        未知語や文章中の空白(MeCabにより消去される)に対しての処理が必要となる
        そのため、各トークンが元の文章のどの位置にあったかを特定しておく
        """
        words = self.word_tokenizer.tokenize(text)
        tokens = []
        tokens_original = []
        for word in words:
            # 単語をサブワードに分割しlistに格納する
            tokens_subword = self.subword_tokenizer.tokenize(word)
            tokens.extend(tokens_subword)
            # 未知語対応
            if tokens_subword[0] == "[UNK]":
                tokens_original.append(word)
            else:
                tokens_original.extend(
                    [token.replace("##", "") for token in tokens_subword]
                )

        # トークンが文章中のどの位置にあるかを走査する
        position = 0
        spans = []
        for token in tokens_original:
            length = len(token)
            # トークンの長さにトークンと同一のstrが収まらなければ1つずつずらしていく
            while True:
                if token != text[position: position+length]:
                    position += 1
                else:
                    spans.append([position, position+length])
                    position += length
                    break

        # トークンをID化する
        input_ids = self.convert_tokens_to_ids(tokens)
        # トークンIDを符号化する
        # padding引数とtruncation引数はメソッドのmax_length引数次第で
        encoding = self.prepare_for_model(
            input_ids,
            max_length=max_length,
            padding='max_length' if max_length else False,
            truncation=True if max_length else False
        )

        # 符号化した文章の長さ
        sequence_length = len(encoding['input_ids'])
        # [CLS]用のダミーspanを追加
        # 符号化する際[CLS]と[SEP]トークンを追加されているため -2 する
        # spansとspans[:sequence_length - 2]はイコールでは？
        spans = [[-1, -1]] + spans[:sequence_length - 2]
        # [SEP]、[PAD]用のダミーspanを追加
        # 後ろにダミーspanを追加するだけ
        spans = spans + [[-1, -1]] * (sequence_length - len(spans))

        # 引数に応じてtorch.Tensor型に変換
        if return_tensors == "pt":
            # 二次元のテンソルに変換しないと先で読み込んでくれない
            encoding = {key: torch.tensor([value]) for key, value in encoding.items()}

        return (encoding, spans)

    def convert_bert_output_to_entities(
        self,
        text: str,
        labels_arg: list,
        spans_arg: list,
    ) -> list:
        """
        文章、ラベル列の予測値、各トークンの位置から固有表現を得る
        """

        # labels_arg、spans_argから特殊トークンに対応する部分を取り除く
        labels = []
        spans = []
        for label, span in zip(labels_arg, spans_arg):
            if span[0] != -1:
                labels.append(label)
                spans.append(span)

        # 同じラベルが連続するトークンをまとめて固有表現を抽出する
        entities = []
        for label, group in itertools.groupby(enumerate(labels), key=lambda x: x[1]):
            # label: labelsの要素が連続して同値なものがひとつの要素として順に与えられる
            #        例) labels = [0, 0, 1, 1, 0] -> 0, 1, 0の順で回る
            # group: labelsの要素が連続して同値なものがひとつのイテレータとして与えられる
            #        各イテレータの中身は(enumerate()で与えられるi, labelsの要素)となる
            #        例) labels = [0, 0, 1, 1, 0] ->
            #            [(0, 0), (1, 0)] / [(2, 1), (3, 1)] / [(4, 0)]
            group = list(group)

            # groupの最初と最後の要素に含まれるlabelに相当する部分をspansから取得
            # spans = [[0, 1], [1, 2]]
            # group = [(0, 1), (1, 1)]
            # -> (0, 2)
            start = spans[group[0][0]][0]
            end = spans[group[-1][0]][1]

            # labelがでない(=固有表現)のときentityを追加する
            if label != 0:
                entity = {
                    "name": text[start:end],
                    "span": [start, end],
                    "type_id": label,
                }
                entities.append(entity)
        return entities

In [5]:
# 8-6

model_name = 'cl-tohoku/bert-base-japanese-whole-word-masking'
tokenizer = NER_tokenizer.from_pretrained(model_name)

The tokenizer class you load from this checkpoint is not the same type as the class this function is called from. It may result in unexpected tokenization. 
The tokenizer class you load from this checkpoint is 'BertJapaneseTokenizer'. 
The class this function is called from is 'NER_tokenizer'.


実装した各メソッドの動作を確認する。

In [6]:
# 8-7

text = '昨日のみらい事務所との打ち合わせは順調だった。'
entities = [
    {'name': 'みらい事務所', 'span': [3,9], 'type_id': 1}
]

encoding = tokenizer.encode_plus_tagged(
    text, entities, max_length=20
)
print(encoding)

{'input_ids': [2, 10271, 28486, 5, 546, 10780, 2464, 13, 5, 1878, 2682, 9, 10750, 308, 10, 8, 3, 0, 0, 0], 'token_type_ids': [0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0], 'attention_mask': [1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 0, 0, 0], 'labels': [0, 0, 0, 0, 1, 1, 1, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0]}


In [7]:
# 8-8

text = '騰訊の英語名はTencent Holdings Ltdである。'
encoding, spans = tokenizer.encode_plus_untagged(
    text, return_tensors='pt'
)
print(f"encoding: {encoding}")
print(f"spans: {spans}")

encoding: {'input_ids': tensor([[    2,     1, 26280,     5,  1543,   125,     9,  6749, 28550,  2953,
         28550, 28566, 21202, 28683, 14050, 12475,    12,    31,     8,     3]]), 'token_type_ids': tensor([[0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0]]), 'attention_mask': tensor([[1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1]])}
spans: [[-1, -1], [0, 1], [1, 2], [2, 3], [3, 5], [5, 6], [6, 7], [7, 9], [9, 10], [10, 12], [12, 13], [13, 14], [15, 18], [18, 19], [19, 23], [24, 27], [27, 28], [28, 30], [30, 31], [-1, -1]]


In [8]:
# 8-9

labels_predicted = [
    0, 1, 1, 0, 0, 0, 0, 1, 1, 1, 1, 1, 1, 1, 1, 1, 0, 0, 0, 0
]
entities = tokenizer.convert_bert_output_to_entities(
    text, labels_predicted, spans
)
print(entities)

[{'name': '騰訊', 'span': [0, 2], 'type_id': 1}, {'name': 'Tencent Holdings Ltd', 'span': [7, 27], 'type_id': 1}]


### 8.5.3 BERTによる固有表現抽出


固有表現抽出は、与えられた文章をトークン化し、そのラベルを予測する分類問題として扱うことができる。Transformersでは`transformers.BertForTokenClassifiaction`クラスが提供されている。IO方式の場合、`num_labels`引数には`ラベルの数 + 1`を渡す(ラベルなし`0`をカウントするため)。

In [9]:
# 8-10

model_name = 'cl-tohoku/bert-base-japanese-whole-word-masking'
tokenizer = NER_tokenizer.from_pretrained(model_name)
model = BertForTokenClassification.from_pretrained(model_name, num_labels=4)
model = model.cuda()

The tokenizer class you load from this checkpoint is not the same type as the class this function is called from. It may result in unexpected tokenization. 
The tokenizer class you load from this checkpoint is 'BertJapaneseTokenizer'. 
The class this function is called from is 'NER_tokenizer'.
Some weights of the model checkpoint at cl-tohoku/bert-base-japanese-whole-word-masking were not used when initializing BertForTokenClassification: ['cls.seq_relationship.weight', 'cls.predictions.transform.LayerNorm.weight', 'cls.seq_relationship.bias', 'cls.predictions.transform.dense.bias', 'cls.predictions.transform.dense.weight', 'cls.predictions.decoder.weight', 'cls.predictions.transform.LayerNorm.bias', 'cls.predictions.bias']
- This IS expected if you are initializing BertForTokenClassification from the checkpoint of a model trained on another task or with another architecture (e.g. initializing a BertForSequenceClassification model from a BertForPreTraining model).
- This IS NOT expecte

`transformers.BertForTokenClassification`も、学習時と推論時のふたつの動作がある。

- 学習時: 符号化した文章とトークンごとのラベルを入力として受け付け、損失の値を出力する。
- 推論時: 符号化した文章を入力として受け付け、トークンごとのラベルの分類スコアを出力する。

以下のような文章を考える。

|文章|固有表現|
|:-|:-|
|AさんはB大学に入学した。|(A, 人名), (B大学, 組織名)|
|CDE株式会社は新製品「E」を販売する。|(CDE株式会社, 組織名), (E, 製品名)|

推論時は、符号化された文章をBERTに入力することで、トークンごとのラベルの分類スコアを得る。<br>



In [10]:
# 8-11

text = "AさんはB大学に入学しました。"

# 符号化とトークンの文章位置の抽出
encoding, spans = tokenizer.encode_plus_untagged(
    text, return_tensors='pt'
)
encoding = {key: value.cuda() for key, value in encoding.items()}

In [11]:
# modelによる推論
with torch.no_grad():
    output = model(**encoding)
scores = output.logits

# 1つ目の文章の各トークン12コに対する予測値の二次元テンソル(12, 4)の
# axis=1での最大の要素のインデックス値に置き換える -> (12,)
labels_pred = scores[0].argmax(axis=1).cpu().numpy().tolist()

In [12]:
# モデルの予測値とインデックスから固有表現を抽出する
entities = tokenizer.convert_bert_output_to_entities(
    text, labels_pred, spans
)

In [13]:
entities

[{'name': 'A', 'span': [0, 1], 'type_id': 2},
 {'name': 'さんはB大学に', 'span': [1, 8], 'type_id': 1},
 {'name': '入学', 'span': [8, 10], 'type_id': 2},
 {'name': 'し', 'span': [10, 11], 'type_id': 1},
 {'name': 'まし', 'span': [11, 13], 'type_id': 2},
 {'name': 'た', 'span': [13, 14], 'type_id': 3},
 {'name': '。', 'span': [14, 15], 'type_id': 1}]

ファインチューニングを行っていないため出力`entities`やそのラベル`type_id`は間違っているが、本文から固有表現を抽出することができた。

次に学習時の挙動を確認する。学習時には、符号化された文章と固有表現をラベル列に変換したものをBERTに入力して損失を計算する。損失には`Cross Entropy Loss`が用いられる。<br>
`transformers.BertForTokenClassification`は、入力にラベル列`labels`を含めると損失`loss`を出力する。

In [14]:
# 8-12

data = [
    {
        'text': 'AさんはB大学に入学した。',
        'entities': [
            {'name': 'A', 'span': [0, 1], 'type_id': 2},
            {'name': 'B大学', 'span': [4, 7], 'type_id': 1}
        ]
    },
    {
        'text': 'CDE株式会社は新製品「E」を販売する。',
        'entities': [
            {'name': 'CDE株式会社', 'span': [0, 7], 'type_id': 1},
            {'name': 'E', 'span': [12, 13], 'type_id': 3}
        ]
    }
]

In [15]:
max_length = 32
dataset_for_loader = []
for sample in data:
    text = sample['text']
    entities = sample['entities']
    encoding = tokenizer.encode_plus_tagged(
        text=text, entities=entities, max_length=max_length
    )
    encoding = {key: torch.tensor(value) for key, value in encoding.items()}
    dataset_for_loader.append(encoding)
dataloader = DataLoader(dataset_for_loader, batch_size=len(data))

In [16]:
for batch in dataloader:
    batch = {key: value.cuda() for key, value in batch.items()}
    output = model(**batch)
    loss = output.loss

In [17]:
loss

tensor(1.4556, device='cuda:0', grad_fn=<NllLossBackward0>)

## 8.6 Wikipediaデータセットを用いた固有表現抽出のファインチューニング

ストックマーク社が公開している日本語版Wikipediaから作成された固有表現抽出データセット用いる。

|タイプ|ID|固有表現数|備考|
|:--|:--:|:--:|:--|
|人名|1|2980||
|法人名|2|2485|法人または法人に類する組織|
|政治的組織名|3|1180|政治的組織名、政党名、政府組織名、行政組織名、軍隊名、国際組織名|
|その他の組織名|4|1051|競技組織名、公演組織名、その他|
|地名|5|2157||
|施設名|6|1108||
|製品名|7|1215|商品名、番組名、映画名、書籍名、歌名、ブランド名等|
|イベント名|8|1009||

まずはデータをダウンロードする。

In [18]:
import os
import itertools
import random
import json
from tqdm import tqdm
import numpy as np
import unicodedata
from sklearn.metrics import accuracy_score

import torch
from torch.utils.data import DataLoader
from transformers import BertJapaneseTokenizer, BertForTokenClassification
import pytorch_lightning as pl

In [19]:
# 8-13

os.makedirs("data/chapter8", exist_ok=True)

In [20]:
!curl -o "data/chapter8/ner.json" "https://raw.githubusercontent.com/stockmarkteam/ner-wikipedia-dataset/main/ner.json"

  % Total    % Received % Xferd  Average Speed   Time    Time     Time  Current
                                 Dload  Upload   Total   Spent    Left  Speed
100 3947k  100 3947k    0     0  1533k      0  0:00:02  0:00:02 --:--:-- 1533k


<br>

データをロードし整形する。

In [21]:
# 8-14

dataset = json.load(open("data/chapter8/ner.json"))

# 固有表現のタイプとIDの辞書を定義する
type_id_dict = {
    "人名": 1,
    "法人名": 2,
    "政治的組織名": 3,
    "その他の組織名": 4,
    "地名": 5,
    "施設名": 6,
    "製品名": 7,
    "イベント名": 8
}

# typeの日本語名をtype_idに変更
# 文字列の正規化
for sample in dataset:
    sample['text'] = unicodedata.normalize('NFKC', sample['text'])
    for entity in sample['entities']:
        entity['type_id'] = type_id_dict[entity['type']]
        del entity['type']

In [22]:
dataset[0]

{'curid': '3572156',
 'text': 'SPRiNGSと最も仲の良いライバルグループ。',
 'entities': [{'name': 'SPRiNGS', 'span': [0, 7], 'type_id': 4}]}

In [23]:
# データセットの分割

random.shuffle(dataset)
n = len(dataset)
n_train = int(n * 0.6)
n_val = int(n * 0.2)

train_dataset = dataset[:n_train]
val_dataset = dataset[n_train: n_train + n_val]
test_dataset = dataset[n_train + n_val:]

## 8.7 ファインチューニング

`train`、`val`データに対するデータローダを作成する。

In [24]:
# 8-14

def modify_dataset_for_loader(tokenizer, dataset, max_length):
    """
    データセットをデータローダに入力できるように整形
    """
    dataset_for_loader = []
    for sample in dataset:
        text = sample['text']
        entities = sample['entities']
        encoding = tokenizer.encode_plus_tagged(
            text, entities, max_length
        )
        encoding = {key: torch.tensor(value) for key, value in encoding.items()}
        dataset_for_loader.append(encoding)
    return dataset_for_loader

In [25]:
# トークナイザの呼び出し
model_name = 'cl-tohoku/bert-base-japanese-whole-word-masking'
tokenizer = NER_tokenizer.from_pretrained(model_name)

# データセットの作成
max_length = 128
train_dataset_for_loader = modify_dataset_for_loader(
    tokenizer, train_dataset, max_length
)
val_dataset_for_loader = modify_dataset_for_loader(
    tokenizer, val_dataset, max_length
)

train_dataloader = DataLoader(
    train_dataset_for_loader, batch_size=32, shuffle=True
)
val_dataloader = DataLoader(
    val_dataset_for_loader, batch_size=256
)

The tokenizer class you load from this checkpoint is not the same type as the class this function is called from. It may result in unexpected tokenization. 
The tokenizer class you load from this checkpoint is 'BertJapaneseTokenizer'. 
The class this function is called from is 'NER_tokenizer'.


<br>

Pytorch Lightningでファインチューニングを行う。

In [26]:
# 8-16

class BertForTokenClassification_pl(pl.LightningModule):
    def __init__(
        self,
        model_name: str,
        num_labels: int, # ラベル数+1
        lr: float
    ):
        super().__init__()
        self.save_hyperparameters()
        self.model = BertForTokenClassification.from_pretrained(
            model_name, num_labels=num_labels
        )

    def training_step(self, batch, batch_idx):
        output = self.model(**batch)
        loss = output.loss
        self.log('train_loss', loss)
        return loss

    def validation_step(self, batch, batch_idx):
        output = self.model(**batch)
        loss = output.loss
        self.log('val_loss', loss)

    def configure_optimizers(self):
        return torch.optim.Adam(self.parameters(), lr=self.hparams.lr)

In [29]:
# checkpointの設定
checkpoint = pl.callbacks.ModelCheckpoint(
    monitor='val_loss',
    mode='min',
    save_top_k=1,
    save_weights_only=True,
    dirpath="/content/model"
)
trainer = pl.Trainer(
    devices=1,
    accelerator='gpu',
    max_epochs=5,
    callbacks=[checkpoint],
)

# ファインチューニング
model = BertForTokenClassification_pl(
    model_name,
    num_labels=9, # 全8ラベル+1にする
    lr=1e-5
)
trainer.fit(model, train_dataloader, val_dataloader)

GPU available: True (cuda), used: True
TPU available: False, using: 0 TPU cores
IPU available: False, using: 0 IPUs
HPU available: False, using: 0 HPUs


Some weights of the model checkpoint at cl-tohoku/bert-base-japanese-whole-word-masking were not used when initializing BertForTokenClassification: ['cls.seq_relationship.weight', 'cls.predictions.transform.LayerNorm.weight', 'cls.seq_relationship.bias', 'cls.predictions.transform.dense.bias', 'cls.predictions.transform.dense.weight', 'cls.predictions.decoder.weight', 'cls.predictions.transform.LayerNorm.bias', 'cls.predictions.bias']
- This IS expected if you are initializing BertForTokenClassification from the checkpoint of a model trained on another task or with another architecture (e.g. initializing a BertForSequenceClassification model from a BertForPreTraining model).
- This IS NOT expected if you are initializing BertForTokenClassification from the checkpoint of a model that you expect to be exactly identical (initializing a BertForSequenceClassification model from a BertForSequenceClassification model).
Some weights of BertForTokenClassification were not initialized from the m

Sanity Checking: 0it [00:00, ?it/s]

Training: 0it [00:00, ?it/s]

Validation: 0it [00:00, ?it/s]

Validation: 0it [00:00, ?it/s]

Validation: 0it [00:00, ?it/s]

Validation: 0it [00:00, ?it/s]

Validation: 0it [00:00, ?it/s]

`Trainer.fit` stopped: `max_epochs=5` reached.


In [30]:
best_model_path = checkpoint.best_model_path

## 8.8 固有表現抽出の性能評価

固有表現抽出の性能評価は、適合率、再現率、F1-Scoreで行う。

> Aさんは2000年にB大学に入学した。<br>
> 正解ラベル：`(A, 人名), (2000年, 時間), (B大学, 組織名)`

という文章を予測したモデルが`(A, 人名), (入学, 組織名)`と予測した場合、

- 適合率：予測結果に対し`1/2`で正解なので`0.5`
- 再現率：正解ラベルに対し`1/3`で正解なので`0.33...`
- F1-Score：適合率と再現率の調和平均のため`2 * 適合率 * 再現率 / (適合率 + 再現率) = 0.4`

となる。計算は、各ラベル種別ごとに行うことも可能。

まず、ファインチューニングしたBERTモデルで、テストデータを予測し固有表現抽出を行う。

### 8.8.1 予測のバッチ処理

In [132]:
# 8-17

tokenizer = NER_tokenizer.from_pretrained(model_name)

# モデルを呼び出してGPUメモリに載せる
model = BertForTokenClassification_pl.load_from_checkpoint(best_model_path)
model = model.model.cuda()

# 固有表現抽出
batch_size = 32 # batch_sizeの大きさは適当に
max_length = 128 # バッチ内のテンソルのサイズを同一にする
entities_test_lst = []
entities_pred_lst = []
for i in range(0, len(test_dataset), batch_size): # バッチ処理
    samples_batch = test_dataset[i: i+batch_size]
    
    # バッチにわけたデータを符号化する
    text_batch = []
    encoding_batch = []
    spans_batch = []
    for sample in samples_batch:
        text = sample['text']
        encoding, spans = tokenizer.encode_plus_untagged(
            text, max_length=max_length, return_tensors='pt'
        )
        text_batch.append(text)
        encoding_batch.append(encoding)
        spans_batch.append(spans)
        
        entities_test_lst.append(sample['entities'])

    # encoding_samplesのvalueをkeyでまとめなおす
    encoding_batch_modified = {}
    for key in encoding_batch[0].keys():
        encoding_batch_modified[key] = \
            torch.vstack([encoding[key] for encoding in encoding_batch]).cuda()

    # バッチごと予測
    with torch.no_grad():
        output = model(**encoding_batch_modified)
    scores = output.logits

    # 固有表現に変換
    labels_pred_batch = scores.argmax(axis=2).cpu().numpy()
    for text, labels_pred, spans in zip(text_batch, labels_pred_batch, spans_batch):
        entities_pred = tokenizer.convert_bert_output_to_entities(
            text, labels_pred, spans
        )
        entities_pred_lst.append(entities_pred)

The tokenizer class you load from this checkpoint is not the same type as the class this function is called from. It may result in unexpected tokenization. 
The tokenizer class you load from this checkpoint is 'BertJapaneseTokenizer'. 
The class this function is called from is 'NER_tokenizer'.
Some weights of the model checkpoint at cl-tohoku/bert-base-japanese-whole-word-masking were not used when initializing BertForTokenClassification: ['cls.seq_relationship.weight', 'cls.predictions.transform.LayerNorm.weight', 'cls.seq_relationship.bias', 'cls.predictions.transform.dense.bias', 'cls.predictions.transform.dense.weight', 'cls.predictions.decoder.weight', 'cls.predictions.transform.LayerNorm.bias', 'cls.predictions.bias']
- This IS expected if you are initializing BertForTokenClassification from the checkpoint of a model trained on another task or with another architecture (e.g. initializing a BertForSequenceClassification model from a BertForPreTraining model).
- This IS NOT expecte

In [138]:
# 結果表示
print(f"label: {entities_test_lst[0]}")
print(f"label pred: {entities_pred_lst[0]}")

label: [{'name': '新日本石油', 'span': [0, 5], 'type_id': 2}, {'name': 'JX日鉱日石開発', 'span': [27, 35], 'type_id': 2}, {'name': '石油鉱業連盟', 'span': [43, 49], 'type_id': 2}]
label pred: [{'name': '新日本石油', 'span': [0, 5], 'type_id': 2}, {'name': 'JX日鉱日石開発', 'span': [27, 35], 'type_id': 2}, {'name': '石油', 'span': [43, 45], 'type_id': 2}, {'name': '鉱業連盟', 'span': [45, 49], 'type_id': 3}]


### 8.8.2 性能評価

In [ ]:
# 8-19
def evaluate_model(entities_list, entities_predicted_list, type_id=None):
    """
    正解と予測を比較し、モデルの固有表現抽出の性能を評価する。
    type_idがNoneのときは、全ての固有表現のタイプに対して評価する。
    type_idが整数を指定すると、その固有表現のタイプのIDに対して評価を行う。
    """
    num_entities = 0 # 固有表現(正解)の個数
    num_predictions = 0 # BERTにより予測された固有表現の個数
    num_correct = 0 # BERTにより予測のうち正解であった固有表現の数

    # それぞれの文章で予測と正解を比較。
    # 予測は文章中の位置とタイプIDが一致すれば正解とみなす。
    for entities, entities_predicted \
        in zip(entities_list, entities_predicted_list):

        if type_id:
            entities = [ e for e in entities if e['type_id'] == type_id ]
            entities_predicted = [
                e for e in entities_predicted if e['type_id'] == type_id
            ]

        get_span_type = lambda e: (e['span'][0], e['span'][1], e['type_id'])
        set_entities = set( get_span_type(e) for e in entities )
        set_entities_predicted = \
            set( get_span_type(e) for e in entities_predicted )

        num_entities += len(entities)
        num_predictions += len(entities_predicted)
        num_correct += len( set_entities & set_entities_predicted )

    # 指標を計算
    precision = num_correct/num_predictions # 適合率
    recall = num_correct/num_entities # 再現率
    f_value = 2*precision*recall/(precision+recall) # F値

    result = {
        'num_entities': num_entities,
        'num_predictions': num_predictions,
        'num_correct': num_correct,
        'precision': precision,
        'recall': recall,
        'f_value': f_value
    }

    return result

In [ ]:
# 8-20
print( evaluate_model(entities_list, entities_predicted_list) )

In [ ]:
# 8-21
class NER_tokenizer_BIO(BertJapaneseTokenizer):

    # 初期化時に固有表現のカテゴリーの数`num_entity_type`を
    # 受け入れるようにする。
    def __init__(self, *args, **kwargs):
        self.num_entity_type = kwargs.pop('num_entity_type')
        super().__init__(*args, **kwargs)

    def encode_plus_tagged(self, text, entities, max_length):
        """
        文章とそれに含まれる固有表現が与えられた時に、
        符号化とラベル列の作成を行う。
        """
        # 固有表現の前後でtextを分割し、それぞれのラベルをつけておく。
        splitted = [] # 分割後の文字列を追加していく
        position = 0
        for entity in entities:
            start = entity['span'][0]
            end = entity['span'][1]
            label = entity['type_id']
            splitted.append({'text':text[position:start], 'label':0})
            splitted.append({'text':text[start:end], 'label':label})
            position = end
        splitted.append({'text': text[position:], 'label':0})
        splitted = [ s for s in splitted if s['text'] ]

        # 分割されたそれぞれの文章をトークン化し、ラベルをつける。
        tokens = [] # トークンを追加していく
        labels = [] # ラベルを追加していく
        for s in splitted:
            tokens_splitted = self.tokenize(s['text'])
            label = s['label']
            if label > 0: # 固有表現
                # まずトークン全てにI-タグを付与
                labels_splitted =  \
                    [ label + self.num_entity_type ] * len(tokens_splitted)
                # 先頭のトークンをB-タグにする
                labels_splitted[0] = label
            else: # それ以外
                labels_splitted =  [0] * len(tokens_splitted)

            tokens.extend(tokens_splitted)
            labels.extend(labels_splitted)

        # 符号化を行いBERTに入力できる形式にする。
        input_ids = self.convert_tokens_to_ids(tokens)
        encoding = self.prepare_for_model(
            input_ids,
            max_length=max_length,
            padding='max_length',
            truncation=True
        )

        # ラベルに特殊トークンを追加
        labels = [0] + labels[:max_length-2] + [0]
        labels = labels + [0]*( max_length - len(labels) )
        encoding['labels'] = labels

        return encoding

    def encode_plus_untagged(
        self, text, max_length=None, return_tensors=None
    ):
        """
        文章をトークン化し、それぞれのトークンの文章中の位置も特定しておく。
        IO法のトークナイザのencode_plus_untaggedと同じ
        """
        # 文章のトークン化を行い、
        # それぞれのトークンと文章中の文字列を対応づける。
        tokens = [] # トークンを追加していく。
        tokens_original = [] # トークンに対応する文章中の文字列を追加していく。
        words = self.word_tokenizer.tokenize(text) # MeCabで単語に分割
        for word in words:
            # 単語をサブワードに分割
            tokens_word = self.subword_tokenizer.tokenize(word)
            tokens.extend(tokens_word)
            if tokens_word[0] == '[UNK]': # 未知語への対応
                tokens_original.append(word)
            else:
                tokens_original.extend([
                    token.replace('##','') for token in tokens_word
                ])

        # 各トークンの文章中での位置を調べる。（空白の位置を考慮する）
        position = 0
        spans = [] # トークンの位置を追加していく。
        for token in tokens_original:
            l = len(token)
            while 1:
                if token != text[position:position+l]:
                    position += 1
                else:
                    spans.append([position, position+l])
                    position += l
                    break

        # 符号化を行いBERTに入力できる形式にする。
        input_ids = self.convert_tokens_to_ids(tokens)
        encoding = self.prepare_for_model(
            input_ids,
            max_length=max_length,
            padding='max_length' if max_length else False,
            truncation=True if max_length else False
        )
        sequence_length = len(encoding['input_ids'])
        # 特殊トークン[CLS]に対するダミーのspanを追加。
        spans = [[-1, -1]] + spans[:sequence_length-2]
        # 特殊トークン[SEP]、[PAD]に対するダミーのspanを追加。
        spans = spans + [[-1, -1]] * ( sequence_length - len(spans) )

        # 必要に応じてtorch.Tensorにする。
        if return_tensors == 'pt':
            encoding = { k: torch.tensor([v]) for k, v in encoding.items() }

        return encoding, spans

    @staticmethod
    def Viterbi(scores_bert, num_entity_type, penalty=10000):
        """
        Viterbiアルゴリズムで最適解を求める。
        """
        m = 2*num_entity_type + 1
        penalty_matrix = np.zeros([m, m])
        for i in range(m):
            for j in range(1+num_entity_type, m):
                if not ( (i == j) or (i+num_entity_type == j) ):
                    penalty_matrix[i,j] = penalty

        path = [ [i] for i in range(m) ]
        scores_path = scores_bert[0] - penalty_matrix[0,:]
        scores_bert = scores_bert[1:]

        for scores in scores_bert:
            assert len(scores) == 2*num_entity_type + 1
            score_matrix = np.array(scores_path).reshape(-1,1) \
                + np.array(scores).reshape(1,-1) \
                - penalty_matrix
            scores_path = score_matrix.max(axis=0)
            argmax = score_matrix.argmax(axis=0)
            path_new = []
            for i, idx in enumerate(argmax):
                path_new.append( path[idx] + [i] )
            path = path_new

        labels_optimal = path[np.argmax(scores_path)]
        return labels_optimal

    def convert_bert_output_to_entities(self, text, scores, spans):
        """
        文章、分類スコア、各トークンの位置から固有表現を得る。
        分類スコアはサイズが（系列長、ラベル数）の2次元配列
        """
        assert len(spans) == len(scores)
        num_entity_type = self.num_entity_type

        # 特殊トークンに対応する部分を取り除く
        scores = [score for score, span in zip(scores, spans) if span[0]!=-1]
        spans = [span for span in spans if span[0]!=-1]

        # Viterbiアルゴリズムでラベルの予測値を決める。
        labels = self.Viterbi(scores, num_entity_type)

        # 同じラベルが連続するトークンをまとめて、固有表現を抽出する。
        entities = []
        for label, group \
            in itertools.groupby(enumerate(labels), key=lambda x: x[1]):

            group = list(group)
            start = spans[group[0][0]][0]
            end = spans[group[-1][0]][1]

            if label != 0: # 固有表現であれば
                if 1 <= label <= num_entity_type:
                     # ラベルが`B-`ならば、新しいentityを追加
                    entity = {
                        "name": text[start:end],
                        "span": [start, end],
                        "type_id": label
                    }
                    entities.append(entity)
                else:
                    # ラベルが`I-`ならば、直近のentityを更新
                    entity['span'][1] = end
                    entity['name'] = text[entity['span'][0]:entity['span'][1]]

        return entities

In [ ]:
# 8-22
# トークナイザのロード
# 固有表現のカテゴリーの数`num_entity_type`を入力に入れる必要がある。
tokenizer = NER_tokenizer_BIO.from_pretrained(
    MODEL_NAME,
    num_entity_type=8
)

# データセットの作成
max_length = 128
dataset_train_for_loader = create_dataset(
    tokenizer, dataset_train, max_length
)
dataset_val_for_loader = create_dataset(
    tokenizer, dataset_val, max_length
)

# データローダの作成
dataloader_train = DataLoader(
    dataset_train_for_loader, batch_size=32, shuffle=True
)
dataloader_val = DataLoader(dataset_val_for_loader, batch_size=256)

In [ ]:
# 8-23

# ファインチューニング
checkpoint = pl.callbacks.ModelCheckpoint(
    monitor='val_loss',
    mode='min',
    save_top_k=1,
    save_weights_only=True,
    dirpath='model_BIO/'
)

trainer = pl.Trainer(
    gpus=1,
    max_epochs=5,
    callbacks=[checkpoint]
)

# PyTorch Lightningのモデルのロード
num_entity_type = 8
num_labels = 2*num_entity_type+1
model = BertForTokenClassification_pl(
    MODEL_NAME, num_labels=num_labels, lr=1e-5
)

# ファインチューニング
trainer.fit(model, dataloader_train, dataloader_val)
best_model_path = checkpoint.best_model_path

# 性能評価
model = BertForTokenClassification_pl.load_from_checkpoint(
    best_model_path
)
bert_tc = model.bert_tc.cuda()

entities_list = [] # 正解の固有表現を追加していく
entities_predicted_list = [] # 抽出された固有表現を追加していく
for sample in tqdm(dataset_test):
    text = sample['text']
    encoding, spans = tokenizer.encode_plus_untagged(
        text, return_tensors='pt'
    )
    encoding = { k: v.cuda() for k, v in encoding.items() }

    with torch.no_grad():
        output = bert_tc(**encoding)
        scores = output.logits
        scores = scores[0].cpu().numpy().tolist()

    # 分類スコアを固有表現に変換する
    entities_predicted = tokenizer.convert_bert_output_to_entities(
        text, scores, spans
    )

    entities_list.append(sample['entities'])
    entities_predicted_list.append( entities_predicted )

print(evaluate_model(entities_list, entities_predicted_list))